In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn import set_config
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, f1_score, confusion_matrix, recall_score


In [2]:
df = pd.read_csv("Loan.csv")

In [3]:
df.head()

,Gender,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area,Loan_Status
0,Male,0,Graduate,No,5849,NaN,1.0,Urban,Y
1,Male,1,Graduate,No,4583,128.0,1.0,Rural,N
2,Male,0,Graduate,Yes,3000,66.0,1.0,Urban,Y
3,Male,0,Not Graduate,No,2583,120.0,1.0,Urban,Y
4,Male,0,Graduate,No,6000,141.0,1.0,Urban,Y


In [4]:
df.isna().sum()

Gender             13
Dependents         15
Education           0
Self_Employed      32
ApplicantIncome     0
LoanAmount         22
Credit_History     50
Property_Area       0
Loan_Status         0
dtype: int64

In [5]:
df.isnull().mean()*100

Gender             2.117264
Dependents         2.442997
Education          0.000000
Self_Employed      5.211726
ApplicantIncome    0.000000
LoanAmount         3.583062
Credit_History     8.143322
Property_Area      0.000000
Loan_Status        0.000000
dtype: float64

In [6]:
list = [col for col in df.columns if df[col].isnull().mean() <0.03 and df[col].isnull().mean()>0]
list

['Gender', 'Dependents']

In [7]:
df.dropna(subset = list[0:2], inplace = True)

In [8]:
df.isnull().mean()*100

Gender             0.000000
Dependents         0.000000
Education          0.000000
Self_Employed      5.460751
ApplicantIncome    0.000000
LoanAmount         3.412969
Credit_History     8.361775
Property_Area      0.000000
Loan_Status        0.000000
dtype: float64

In [9]:
imp1=SimpleImputer(strategy="median")
imp2=SimpleImputer(strategy="most_frequent")

In [10]:
df.head(3)

,Gender,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area,Loan_Status
0,Male,0,Graduate,No,5849,NaN,1.0,Urban,Y
1,Male,1,Graduate,No,4583,128.0,1.0,Rural,N
2,Male,0,Graduate,Yes,3000,66.0,1.0,Urban,Y


In [11]:
tr1=ColumnTransformer([
                    ('Median',imp1,['LoanAmount']),
                    ('Mode',imp2,[0, 1, 3, 6]),
                    ],
                     remainder='passthrough')

In [56]:
## Encoding

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

# Define the transformation
tr2 = ColumnTransformer(
    transformers=[
        ('ohe_Gender', OneHotEncoder(handle_unknown='ignore'), [1, 3, 2, 5]),
        ('ordinal_Propertyarea', OrdinalEncoder(categories=[['Rural', 'Urban', 'Semiurban']]), [7]),
        
    ],
    remainder='passthrough')


In [16]:
label_encoder = preprocessing.LabelEncoder()
df["Loan_Status"] = label_encoder.fit_transform(df["Loan_Status"])
df["Loan_Status"].unique()

array([1, 0])

In [17]:
tr3 = ColumnTransformer([
    ('scale',MinMaxScaler(), [0,1,2,3,4,5,6,7,8,9,10,12]),
    ('scale2',StandardScaler(), [11, 13]),
],     remainder='passthrough')

In [18]:
df["Loan_Status"].unique()

array([1, 0])

In [19]:
clf = LogisticRegression()

In [20]:
pipe = Pipeline([
    ('tr1',tr1),
    ('tr2',tr2),
    ('tr3', tr3),
    ('clf',clf)
])

In [22]:
X = df.drop(columns = ["Loan_Status"], axis = 1)

In [23]:
y = df["Loan_Status"]

In [24]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 586, dtype: int64

In [25]:
X

,Gender,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area
0,Male,0,Graduate,No,5849,NaN,1.0,Urban
1,Male,1,Graduate,No,4583,128.0,1.0,Rural
2,Male,0,Graduate,Yes,3000,66.0,1.0,Urban
3,Male,0,Not Graduate,No,2583,120.0,1.0,Urban
4,Male,0,Graduate,No,6000,141.0,1.0,Urban
...,...,...,...,...,...,...,...,...
609,Female,0,Graduate,No,2900,71.0,1.0,Rural
610,Male,3+,Graduate,No,4106,40.0,1.0,Rural
611,Male,1,Graduate,No,8072,253.0,1.0,Urban
612,Male,2,Graduate,No,7583,187.0,1.0,Urban


In [26]:
## Spliting data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
X_train

,Gender,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area
136,Female,0,Graduate,No,4583,84.0,1.0,Rural
158,Male,0,Graduate,NaN,2980,120.0,1.0,Rural
78,Male,3+,Graduate,No,3167,180.0,0.0,Semiurban
455,Male,2,Graduate,No,3859,96.0,1.0,Semiurban
264,Male,0,Graduate,Yes,5166,128.0,1.0,Semiurban
...,...,...,...,...,...,...,...,...
72,Male,0,Graduate,No,3500,81.0,1.0,Semiurban
109,Male,3+,Not Graduate,No,2071,94.0,1.0,Semiurban
279,Female,0,Not Graduate,No,4100,124.0,NaN,Rural
453,Male,0,Graduate,No,2483,90.0,0.0,Rural


In [28]:
y_train

136    0
158    1
78     0
455    1
264    1
      ..
72     1
109    1
279    1
453    1
105    1
Name: Loan_Status, Length: 468, dtype: int64

In [29]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tr1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Median',
                                                  SimpleImputer(strategy='median'),
                                                  ['LoanAmount']),
                                                 ('Mode',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [0, 1, 3, 6])])),
                ('tr2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_Gender',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [1, 3, 2, 5]),
                                                 ('ordinal_Propertyarea',
                                                  OrdinalEncoder(categories=[['Rural',
                        

In [30]:
from sklearn import set_config
set_config(display='diagram')

In [31]:
# Predict, testing Accuracy
y_pred = pipe.predict(X_test)

In [39]:
from sklearn.metrics import accuracy_score
print("Testing Accuracy: ", accuracy_score(y_test,y_pred))

Testing Accuracy:  0.7457627118644068


In [33]:
# Predict for training accuracy
y_pred1 = pipe.predict(X_train)

In [38]:
from sklearn.metrics import accuracy_score
print("Training Accuracy: ",accuracy_score(y_train,y_pred1))

Training Accuracy:  0.8205128205128205


## Cross Validation

In [35]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

array([0.82978723, 0.82978723, 0.81914894, 0.84946237, 0.78494624])

In [40]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
print("Mean after CV=5, the Accuracy is: ", cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())

Mean after CV=5, the Accuracy is:  0.8226264012811713


## Grid Search

In [41]:
# gridsearchcv
param_grid = {
    "clf__penalty": ['l1', 'l2'],
    "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]
}

In [42]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, i

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tr1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Median',
                                                                         SimpleImputer(strategy='median'),
                                                                         ['LoanAmount']),
                                                                        ('Mode',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [0, 1,
                                                                          3,
                                                                          6])])),
                                       ('tr2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_Gender',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         [1, 3,
                                                                          2...
                                                                        ('ordinal_Propertyarea',
                                                                         OrdinalEncoder(categories=[['Rural',
                                                                                                     'Urban',
                                                                                                     'Semiurban']]),
                                                                         [7])])),
                                       ('tr3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          12]),
                                                                        ('scale2',
                                                                         StandardScaler(),
                                                                         [11,
                                                                          13])])),
                                       ('clf', LogisticRegression())]),
             param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'clf__penalty': ['l1', 'l2']},
             scoring='accuracy')

In [43]:
grid.best_score_

0.8226264012811713

In [44]:
grid.best_params_

{'clf__C': 1, 'clf__penalty': 'l2'}

## Pickle File

In [49]:
# export 
import pickle
pickle.dump(clf,open('pipe_logreg.pkl','wb'))

In [50]:
lr_pipe = pickle.load(open('pipe_logreg.pkl','rb'))

In [51]:
X

,Gender,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area
0,Male,0,Graduate,No,5849,NaN,1.0,Urban
1,Male,1,Graduate,No,4583,128.0,1.0,Rural
2,Male,0,Graduate,Yes,3000,66.0,1.0,Urban
3,Male,0,Not Graduate,No,2583,120.0,1.0,Urban
4,Male,0,Graduate,No,6000,141.0,1.0,Urban
...,...,...,...,...,...,...,...,...
609,Female,0,Graduate,No,2900,71.0,1.0,Rural
610,Male,3+,Graduate,No,4106,40.0,1.0,Rural
611,Male,1,Graduate,No,8072,253.0,1.0,Urban
612,Male,2,Graduate,No,7583,187.0,1.0,Urban


In [52]:
test_data =np.array([["Male", 2, "Graduate", "No", 5433, 250.0, 1.0, "Urban" ]],dtype=object).reshape(1,8)

In [53]:
lr_pipe.predict(test_data)

ValueError: could not convert string to float: 'Male'